# Tutorial: ResponseSchema, RouterChain, and TransformChain in LangChain

### Install required libraries

In [6]:
!pip install langchain -q
!pip install openai -q
!pip install python-dotenv -q

### Load Environment Variables

In [8]:
import os
import openai
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

os.environ.get('OPENAI_API_KEY')

print('OpenAI API key loaded:', os.environ.get('OPENAI_API_KEY') is not None)

OpenAI API key loaded: True


### Basic Chain

In [12]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain

Template="""
Interprete the text and evaluate the text.
title: Title of the Movie.
sentiment: Overall sentiment (positive, negative, mixed).
lead_actor: name of the lead actor/actress.
genre: Main genre of the movie (e.g., sci-fi, comedy, drama)
budget: The budget of the movie

# Here's how we want the output to be structured:
Format the output as JSON with the following keys:
title
sentiment
lead_actor
genre
price

# This is where the input will go:
text: {input}
"""

llm = ChatOpenAI()

prompt_template = ChatPromptTemplate.from_template(template=Template)

chain = LLMChain(llm=llm, prompt=prompt_template)

chain.invoke(input="The movie 'Everything Everywhere All at Once' was a mind-bending masterpiece! Michelle Yeoh's performance was simply phenomenal, and the visuals were breathtaking. It's a must-watch for anyone who loves sci-fi with a touch of heart. The cinema ticket price was 5.12 $")


{'input': "The movie 'Everything Everywhere All at Once' was a mind-bending masterpiece! Michelle Yeoh's performance was simply phenomenal, and the visuals were breathtaking. It's a must-watch for anyone who loves sci-fi with a touch of heart. The cinema ticket price was 5.12 $",
 'text': '{\n    "title": "Everything Everywhere All at Once",\n    "sentiment": "positive",\n    "lead_actor": "Michelle Yeoh",\n    "genre": "sci-fi",\n    "price": "$5.12"\n}'}

### ResponseSchema

In [29]:
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

response_schemas = [
    ResponseSchema(name="title", description="Title of the movie."),
    ResponseSchema(name="sentiment", description="Overall sentiment (positive, negative, mixed)."),
    ResponseSchema(name="lead_actor", description="Name of the lead actor/actress."),
    ResponseSchema(name="genre", description="Main genre of the movie (e.g., sci-fi, comedy, drama)."),
    ResponseSchema(name="price", description="Is the cinema ticket price. Use NONE if it is not provided.", type="float")
]

print(response_schemas)

[ResponseSchema(name='title', description='Title of the movie.', type='string'), ResponseSchema(name='sentiment', description='Overall sentiment (positive, negative, mixed).', type='string'), ResponseSchema(name='lead_actor', description='Name of the lead actor/actress.', type='string'), ResponseSchema(name='genre', description='Main genre of the movie (e.g., sci-fi, comedy, drama).', type='string'), ResponseSchema(name='price', description='Is the cinema ticket price. Use NONE if it is not provided.', type='float')]


In [31]:
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

print(format_instructions)

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"title": string  // Title of the movie.
	"sentiment": string  // Overall sentiment (positive, negative, mixed).
	"lead_actor": string  // Name of the lead actor/actress.
	"genre": string  // Main genre of the movie (e.g., sci-fi, comedy, drama).
	"price": float  // Is the cinema ticket price. Use NONE if it is not provided.
}
```


In [43]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate

prompt = ChatPromptTemplate(
    messages = [
        SystemMessagePromptTemplate.from_template(
            "Analyze the following movie review: \n\n{input}\n\n"
            "Just return the JSON, do not add ANYTHING, NO INTERPRETATION!\n"
            "{format_instructions}"
        )
    ],
    input_variable=["input"],
    partial_variables={"format_instructions": format_instructions}
)

In [45]:
input = prompt.format_prompt(
    input="The movie 'Everything Everywhere All at Once' was a mind-bending masterpiece! Michelle Yeoh's performance was simply phenomenal, and the visuals were breathtaking. It's a must-watch for anyone who loves sci-fi with a touch of heart. The cinema ticket price was 5.12 $"
)

output = llm(input.to_messages())

print(output)

content='```json\n{\n\t"title": "Everything Everywhere All at Once",\n\t"sentiment": "positive",\n\t"lead_actor": "Michelle Yeoh",\n\t"genre": "sci-fi",\n\t"price": 5.12\n}\n```' response_metadata={'token_usage': {'completion_tokens': 51, 'prompt_tokens': 212, 'total_tokens': 263}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-3092fc30-9f67-4b23-9863-7e4cc15b1393-0'


In [47]:
json_output = parser.parse(output.content)
print(json_output)

{'title': 'Everything Everywhere All at Once', 'sentiment': 'positive', 'lead_actor': 'Michelle Yeoh', 'genre': 'sci-fi', 'price': 5.12}


### RouterChain in LangChain

In [72]:
from langchain.chains.router import MultiPromptChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

positive_template = """
You are an AI that focuses on the positive side of things. \
Whenever you analyze a text, you look for the positive aspects and highlight them.
Here is the text:
{input}
"""

neutral_template = """
You are an AI that has a neutral perspective. You just provide a balanced analysis of the text, \
not favoring any positive or negative aspects. Here is the text:
{input}
"""

negative_template = """
You are an AI that is designed to find the negative aspects in a text. \
You analyze a text and show the potential downsides. Here is the text:
{input}
"""

prompt_infos = [
    {
        "name":"positive",
        "description":"Good for analyzing positive sentiments",
        "prompt_template":positive_template
    },
    {
        "name":"neutral",
        "description":"Good for analyzing neutral sentiments",
        "prompt_template":neutral_template
    },
    {
        "name":"negative",
        "description":"Good for analyzing negative sentiments",
        "prompt_template":negative_template
    }
]

In [74]:
destination_chains = {}

In [76]:
for p_infos in prompt_infos:
    name = p_infos["name"]
    prompt_template = p_infos["prompt_template"]
    prompt = PromptTemplate(template=prompt_template, input_variables=["input"])
    chain = LLMChain(llm=llm, prompt=prompt)
    destination_chains[name] = chain

In [78]:
destinations = [f"{p['name']}:{p['description']}" for p in prompt_infos]
print(destinations)

['positive:Good for analyzing positive sentiments', 'neutral:Good for analyzing neutral sentiments', 'negative:Good for analyzing negative sentiments']


In [80]:
destination_str = "\n".join(destinations)
print(destination_str)

positive:Good for analyzing positive sentiments
neutral:Good for analyzing neutral sentiments
negative:Good for analyzing negative sentiments


In [82]:
router_template = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destination_str)

In [84]:
router_prompt=PromptTemplate(
    template=router_template,
    input_variables=["input"],
    output_parser=RouterOutputParser()
)

In [86]:
router_chain = LLMRouterChain.from_llm(llm, router_prompt)

In [88]:
chain = MultiPromptChain(
    router_chain=router_chain,
    destination_chains=destination_chains,
    default_chain=destination_chains["neutral"],
    verbose=True
)

In [90]:
chain.invoke("The new iPhone 15 Pro Max boasts incredible camera capabilities, but the price tag of $1599 is a bit steep for many consumers.")



> Entering new MultiPromptChain chain...
positive: {'input': 'The new iPhone 15 Pro Max boasts incredible camera capabilities, but the price tag of $1599 is a bit steep for many consumers.'}
> Finished chain.


{'input': 'The new iPhone 15 Pro Max boasts incredible camera capabilities, but the price tag of $1599 is a bit steep for many consumers.',
 'text': 'The new iPhone 15 Pro Max has amazing camera capabilities, showcasing the continuous innovation in technology. This high-end device offers top-notch features that are sure to impress photography enthusiasts. While the price tag may be considered steep by some, it reflects the premium quality and cutting-edge technology packed into this device.'}

### Transform Chain

### Simple Transform Chain

In [105]:
from langchain.chains import TransformChain
from langchain.llms import OpenAI

def tranform_func(inputs):
    text=inputs["text"]
    transformed_text = text.upper()
    return {"transformed_text": transformed_text}

llm = OpenAI(temperature=0.9)

transform_chain=TransformChain(
    input_variables=["text"],
    output_variables=["transformed_text"],
    transform=transform_func
)

input_text="Hello, world! this is an example of transofrmchain"

result = transform_chain({'text':input_text})

print(result)

{'text': 'Hello, world! this is an example of transofrmchain', 'transformed_text': 'HELLO, WORLD! THIS IS AN EXAMPLE OF TRANSOFRMCHAIN'}


### Advanced Transform Chain

In [121]:
from langchain.chains import TransformChain, SequentialChain, LLMChain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate

# Chain1: Define a chain for summarizing the text using OpenAI
llm = OpenAI(temperature=0.9)
summarize_prompt=PromptTemplate(
    input_variables=['text'],
    template='Summarize the following text:{text}'
)

summarize_chain = LLMChain(
    llm=llm,
    prompt=summarize_prompt,
    output_key='summary'
)
#Chain2: Define a chain for converting the summary to uppercase and appends its length
def transform_func(inputs):
    summary=inputs['summary']
    transformed_summary=summary.upper()
    summary_length=len(summary.split())
    return{
        'transformed_summary':transformed_summary,
        'summary_length':summary_length
    }

transform_chain = TransformChain(
    input_variables=['summary'],
    output_variables=['transformed_summary','summary_length'],
    transform= transform_func
)
#Chain3: Define a chain for translating the summary to another language using OpenAI
translate_prompt=PromptTemplate(
    input_variables=['transformed_summary'],
    template='Translate the following text to French:{transformed_summary}'
)

translate_chain=LLMChain(
    llm=llm,
    prompt=translate_prompt,
    output_key='translated_summary'
)

# Combine the chains into a sequential chain
advanced_chain = SequentialChain(
    input_variables=['text'],
    chains=[
        summarize_chain,
        transform_chain,
        translate_chain
    ],
    output_variables=['transformed_summary','summary_length','translated_summary']
)

input_text = 'LangChain provides a powerful framework for building language model chains.'

result = advanced_chain({'text':input_text})

print(result)

{'text': 'LangChain provides a powerful framework for building language model chains.', 'transformed_summary': '\nLANGCHAIN IS A COMPREHENSIVE FRAMEWORK THAT ENABLES THE CONSTRUCTION OF ROBUST LANGUAGE MODEL CHAINS. ', 'summary_length': 14, 'translated_summary': '\n\nLANGCHAIN EST UNE STRUCTURE COMPLÈTE QUI PERMET LA CONSTRUCTION DE CHAÎNES DE MODÈLES DE LANGUE ROBUSTES.'}
